# Retrieval and Answer Quality Metrics computation using LLM as Judge in IBM watsonx.governance for RAG task

This notebook demonstrates the creation of a Retrieval Augumented Generation (RAG) pattern using watsonx.ai and computations of reference-free Answer Quality metrics, such as **Faithfulness**, **Answer relevance**, reference based **Answer similarity** metric and Retrieval Quality Metrics such as **Context Relevance**, **Retrieval Precision**, **Average Precision**, **Reciprocal Rank**, **Hit Rate** and **Normalized Discounted Cumulative Gain** for the RAG task type. It also identifies the source attribution using watsonx.governance.

- **Faithfulness** measures how faithful the model output or generated text is to the context sent to the LLM input. The faithfulness score is a value between 0 and 1. A value closer to 1 indicates that the output is more faithful - or grounded - and less hallucinated. A value closer to 0 indicates that the output is less faithful and more hallucinated.

- **Answer relevance** measures how relevant the answer or generated text is to the question. This is one of the ways to determine the quality of your model. The answer relevance score is a value between 0 and 1. A value closer to 1 indicates that the answer is more relevant to the given question. A value closer to 0 indicates that the answer is less relevant to the question.

- **Answer similarity** measures how similar the answer or generated text is to the ground truth or reference answer. This is one of the ways to determine the quality of your model. The answer similarity score is a value between 0 and 1. A value closer to 1 indicates that the answer is more similar to the reference value. A value closer to 0 indicates that the answer is less similar to the reference value.

- **Context Relevance** assesses the degree to which the retrieved context is relevant to the question sent to the LLM. This is one of the ways to determine the quality of your retrieval system. The context relevance score is a value between 0 and 1. A value closer to 1 indicates that the context is more relevant to your question in the prompt. A value closer to 0 indicates that the context is less relevant to your question in the prompt.

- **Retrieval Precision** measures the quantity of relevant contexts from the total contexts retrieved. The retrieval precision is a value between 0 and 1. A value of 1 indicates that all the retrieved contexts are relevant. A value of 0 indicates that none of the retrieved contexts are relevant.

- **Average Precision** evaluates whether all the relevant contexts are ranked higher or not. It is the mean of the precision scores of relevant contexts. The average precision is a value between 0 and 1. A value of 1 indicates that all the relevant contexts are ranked higher. A value of 0 indicates that none of the retrieved contexts are relevant.

- **Reciprocal Rank** is the reciprocal of the rank of the first relevant context. The retrieval reciprocal rank is a value between 0 and 1. A value of 1 indicates that the first relevant context is at first position. A value of 0 indicates that none of the relevant contexts are retrieved.

- **Hit Rate** Hit Rate measures whether there is atleast one relevant context among the retrieved contexts. The hit rate value is either 0 or 1. A value of 1 indicates that there is at least one relevant context. A value of 0 indicates that there is no relevant context in the retrieved contexts.

- **Normalized Discounted Cumulative Gain** Normalized Discounted Cumulative Gain or NDCG measures the ranking quality of the retrieved contexts. The ndcg is a value between 0 and 1. A value of 1 indicates that the retrieved contexts are ranked in the correct order.

## Learning goals

- Read data into a vector database
- Initialize foundation model
- Generate RAG responses
- Configure and compute Retrieval and Answer Quality metrics

## Contents

- [Step 1 - Setup](#setup)
- [Step 2 - Read data and store in vector db](#data)
- [Step 3 - Initialize foundational model using watsonx.ai](#model)
- [Step 4 - Generate the retrieval-augmented responses to questions](#predict)
- [Step 5 - Configure the retrieval and answer quality metrics](#config)
- [Step 6 - Compute the retrieval and answer quality metrics](#compute)
- [Step 7 - Display the results](#results)

## Step 1 - Setup <a id="setup"></a>

### Install necessary libraries

In [4]:
!pip install -U "ibm-metrics-plugin~=3.0.9" | tail -n 1
!pip install -U ibm-watson-openscale | tail -n 1
!pip install -U ibm_watsonx_ai | tail -n 1
!pip install langchain==0.3.4 | tail -n 1
!pip install langchain-huggingface==0.1.2 | tail -n 1
!pip install wget | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install chromadb==0.4.13 | tail -n 1
!pip install pydantic | tail -n 1
!pip install langchain-ibm | tail -n 1

import warnings
warnings.filterwarnings("ignore")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.
langchain-huggingface 0.1.2 requires tokenizers>=0.19.1, but you have tokenizers 0.15.2 which is incompatible.
caikit-nlp 0.4.9 requires sentence-transformers<2.4.0,>=2.3.1, but you have sentence-transformers 3.3.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-metrics-plugin 3.0.11 requires transformers~=4.39.3, but you have transformers 4.46.3 which is incompatible.
caikit-nlp 0.4.9 requires sentence-transformers<2.4.0,>=2.3.1, but you have sentence-transformers 3.3.1 which is incompatible.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-metrics-plugin 3.0.11 requires transformers~=4.39.3, but you have transformers 4.46.3 which is incompatible.
caikit-nlp 0.4.9 requires sentence-transformers<2.4.0,>=2.3.1, but you have sentence-transformers 3.3.1 which is incompatible.


**Note**: you may need to restart the kernel to use updated libraries.

### Configure your credentials

In [ ]:
# Cloud credentials
IAM_URL="https://iam.cloud.ibm.com"
DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
CLOUD_API_KEY = "<EDIT THIS>" # YOUR_CLOUD_API_KEY

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY
}

### Configure your project id
Provide the project id to provide the context needed to run the inference against the watsonx.ai model.

***Hint***: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be "Projects / *project name* /". Click on the "*project name*" link, then get the `project_id` from the project's "Manage" tab ("Project -> Manage -> General -> Details").

In [ ]:
project_id = "<EDIT THIS>"

## Step 2 - Read and store data in a vector database <a id="data"></a>

### Read the data

Download the sample "State of the Union" file.

In [3]:
import wget
import os

data = 'state_of_the_union.txt'
url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(data):
    wget.download(url, out=data)

### Prepare the data for the vector database

Take the `state_of_the_union.txt` speech content data and split it into chunks. 

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader(data)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

### Create an embedding function to store the data in a vector database

Embed the chunked data using an open-source embedding model and load it into Chromadb, a vector database.

**Note**: You can also provide a custom embedding function to be used by Chromadb; the performance of Chromadb may differ depending on the embedding model used.

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

/opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Step 3 - Initialize a foundation model using `watsonx.ai`
<a id="model"></a>

IBM watsonx foundation models are among the <a href="https://python.langchain.com/docs/integrations/llms/watsonxllm" target="_blank" rel="noopener no referrer">list of LLM models supported by Langchain</a>. This example shows how to communicate with <a href="https://newsroom.ibm.com/2023-09-28-IBM-Announces-Availability-of-watsonx-Granite-Model-Series,-Client-Protections-for-IBM-watsonx-Models" target="_blank" rel="noopener no referrer">Granite Model Series</a> using <a href="https://python.langchain.com/docs/get_started/introduction" target="_blank" rel="noopener no referrer">Langchain</a>.

### Define a model
Specify a `model_id` that will be used for inferencing:

In [6]:
model_id = "ibm/granite-3-8b-instruct"

### Define the model parameters
Provide a set of model parameters that will influence the result:

In [7]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

### Set LangChain custom LLM wrapper for watsonx model
Initialize the `WatsonxLLM` class from LangChain with defined parameters, and using `ibm/granite-13b-chat-v2`.

In [8]:
from langchain_ibm import WatsonxLLM

watsonx_llm = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

## Step 4 - Generate retrieval-augmented responses to questions
<a id="predict"></a>

### Build a `RetrievalQA` (question answering chain) to automate the RAG task.

In [9]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [10]:
query1 = "What is ARPA-H?"
query2 = "What is the investment of Ford and GM to build electric vehicles?"
query3 = "What is the proposed tax rate for corporations?"
query4 = "What is Intel going to build?"
query5 = "How many new manufacturing jobs are created last year?"
query6 = "How many electric vehicle charging stations are built?"

questions = [query1 , query2, query3, query4, query5, query6]

### Generate retrieval-augmented responses to the questions

In [14]:
responses = []
contexts = []
for query in questions:
    #Retrive relevant context for each question from the vector db
    docs = docsearch.as_retriever().invoke(query)

    context = []
    #Extract the needed information
    for doc in docs:
        context.append(doc.to_json()['kwargs']['page_content'])

    #Capture the context
    contexts.append(context)

    #Run the prompt and get the response
    response = qa.invoke(query)
    responses.append(response.get("result"))

In [15]:
#Print a sample context retrieved for a query 
print(f"Question:{questions[0]}\n context:{contexts[0]}")

Question:What is ARPA-H?
 context:['Last month, I announced our plan to supercharge  \nthe Cancer Moonshot that President Obama asked me to lead six years ago. \n\nOur goal is to cut the cancer death rate by at least 50% over the next 25 years, turn more cancers from death sentences into treatable diseases.  \n\nMore support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded 

In [16]:
#Print the result
for query in questions:
    print(f"{query} \n {responses[questions.index(query)]} \n")

What is ARPA-H? 
 

ARPA-H, or the Advanced Research Projects Agency for Health, is a proposed agency in the United States that would be dedicated to driving breakthroughs in cancer, Alzheimer's, diabetes, and other diseases. It is based on DARPA, the Defense Department project that led to the Internet, GPS, and other technologies. The agency would be funded by Congress as part of the Cancer Moonshot initiative, 

What is the investment of Ford and GM to build electric vehicles? 
 

Ford is investing $11 billion to build electric vehicles, creating 11,000 jobs across the country. GM is making the largest investment in its history—$7 billion to build electric vehicles, creating 4,000 jobs in Michigan. 

What is the proposed tax rate for corporations? 
 

The proposed tax rate for corporations is 15%. 

What is Intel going to build? 
 

Intel is going to build up to eight state-of-the-art factories in one place, which will be a $20 billion semiconductor "mega site". These factories will 

### Construct a dataframe with question, contexts and answer to be used for metrics computation

In [17]:
import pandas as pd
data = pd.DataFrame(contexts, columns=["context1", "context2", "context3", "context4"])
data["question"] = questions
data["answer"] = responses
data

,context1,context2,context3,context4,question,answer
0,"Last month, I announced our plan to supercharg...",We’re also ready with anti-viral treatments. I...,For that purpose we’ve mobilized American grou...,"If you travel 20 miles east of Columbus, Ohio,...",What is ARPA-H?,"\n\nARPA-H, or the Advanced Research Projects ..."
1,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is the investment of Ford and GM to build...,\n\nFord is investing $11 billion to build ele...
2,My plan will cut the cost in half for most fam...,We got more than 130 countries to agree on a g...,And unlike the $2 Trillion tax cut passed in t...,We’re going after the criminals who stole bill...,What is the proposed tax rate for corporations?,\n\nThe proposed tax rate for corporations is ...
3,"If you travel 20 miles east of Columbus, Ohio,...",So let’s not wait any longer. Send it to my de...,When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is Intel going to build?,\n\nIntel is going to build up to eight state-...
4,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,"As Ohio Senator Sherrod Brown says, “It’s time...",How many new manufacturing jobs are created la...,"\n\n369,000 new manufacturing jobs were create..."
5,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",It is going to transform America and put us on...,Vice President Harris and I ran for office wit...,How many electric vehicle charging stations ar...,\n\nThe text does not provide specific informa...


## Step 5 - Configure Retrieval and Answer Quality metrics
<a id="config"></a>

### Parameters

#### Common parameters

| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| context_columns | The list of context column names in the input data frame. |  |  |
| question_column | the name of the question column in the input data frame. |  |  |
| answer_column | The name of the answer column in the input data frame |  |  |
| record_level [Optional] | The flag to return the record level metrics values. Set the flag under configuration to generate record level metrics for all the metrics. Set the flag under specific metric to generate record level metrics for that metric alone. | False | True, False |
| scoring_fn | The scoring function which takes in the prompts input dataframe and score the LLM acting as Judge, return the output as a dataframe. The input data frame will have a single column "prompt" and the output data frame can either have a single column or if there are multiple columns, return the model output text in "generated_text" column. | | |
| language_code[Optional] | The language specific prompt used for computing configured metrics in LLM as judge. | `en` | `en`, `ja` |

### Metric parameters

| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| record_level [Optional] | The flag to return the record level metrics values. Setting the flag under specific metric overrides the value provided at the configuration level. | False | True, False |
| metric_prompt_template [Optional] | The prompt template used to compute the metric value. User can override the prompt template used by watsonx.governance to compute the metric using this parameter. The prompt template should use the variables {context}, {question}, {answer}, {reference_answer} as needed and these variable values will be filled with the actual data while calling the scoring function. The prompt response should return the metric value in the range 1-10 for the respective metric and in one of the formats ["4", "7 star", "star: 8", "stars: 9"] as answer. | | |

### Verify client version

In [18]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY, url="https://iam.cloud.ibm.com")
client = APIClient(authenticator=authenticator, service_url="https://aiopenscale.cloud.ibm.com")

print(client.version)

3.0.41


### Define the scoring function to invoke the LLM acting as Judge while compute the metrics

The scoring function is implemeted using model from watsonx.ai from cloud. The model FLAN_T5_XXL is used as the judge here. The other models which can be used from watsonx.ai are FLAN_UL2, FLAN_T5_XL, MIXTRAL_8X7B_INSTRUCT_V01_Q

The function can be changed as needed to invoke external models as well. The quality of the retrieval and answer quality metrics can vary with the model used as judge.

In [21]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
import pandas as pd

generate_params = {
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.TEMPERATURE: 0.0
}

model = ModelInference(
    model_id=ModelTypes.FLAN_T5_XXL,
    params=generate_params,
    credentials={
        "apikey": credentials.get("apikey"),
        "url": credentials.get("url")
    },
    project_id=project_id
)

def scoring_fn(data):
    results = []

    for prompt_text in data.iloc[:, 0].values.tolist():
        model_response = model.generate_text(prompt=prompt_text)
        results.append(model_response)
    return pd.DataFrame({"generated_text": results})

### Configure context relevance, faithfulness, answer relevance and answer similarity parameters

In [22]:
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMTextMetricGroup, LLMCommonMetrics, LLMRAGMetrics, RetrievalQualityMetric

# Edit below values based on the input data
context_columns = ["context1", "context2", "context3", "context4"]
question_column = "question"
answer_column = "answer"

config_json = {
            "configuration": {
                "context_columns": context_columns,
                "question_column": question_column,
                "scoring_fn": scoring_fn,
                "record_level": True,
                # Uncomment language_code parameter to use other language specific prompts
                # "language_code": "en",
                LLMTextMetricGroup.RAG.value: {
                        LLMRAGMetrics.RETRIEVAL_QUALITY.value: {
                            # RetrievalQualityMetric.CONTEXT_RELEVANCE.value: {
                                #"record_level": True,
                                #"metric_prompt_template": ""
                            #}
                            # RetrievalQualityMetric.RETRIEVAL_PRECISION.value: {},
                            # RetrievalQualityMetric.AVERAGE_PRECISION.value: {},
                            # RetrievalQualityMetric.RECIPROCAL_RANK.value: {},
                            # RetrievalQualityMetric.HIT_RATE.value: {},
                            # RetrievalQualityMetric.NDCG.value: {}
                        },
                        LLMCommonMetrics.FAITHFULNESS.value: {
                            #"record_level": True,
                            #"metric_prompt_template": ""
                        },
                        LLMCommonMetrics.ANSWER_RELEVANCE.value: {
                            #"record_level": True,
                            #"metric_prompt_template": ""
                        },
                        LLMCommonMetrics.ANSWER_SIMILARITY.value: {
                            #"record_level": True,
                            #"metric_prompt_template": ""
                        }
                }
            }
        }

### Create the input, output and reference data frames and send them as input to compute metrics

In [23]:
df_input = pd.DataFrame(data, columns=context_columns+[question_column])
df_output = pd.DataFrame(data, columns=[answer_column])
df_reference = pd.DataFrame(data, columns=[answer_column])

## Step 6 - Compute Retrieval and Answer Quality metrics <a id="compute"></a>

In [24]:
import json
metrics_result = client.llm_metrics.compute_metrics(config_json, 
                                                    sources=df_input, 
                                                    predictions=df_output,
                                                    references=df_reference)

print(json.dumps(metrics_result, indent=2))

[nltk_data] Downloading package punkt_tab to /home/wsuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


{
  "answer_relevance": {
    "metric_value": 0.7667,
    "mean": 0.7667,
    "min": 0.4,
    "max": 1.0,
    "total_records": 6,
    "record_level_metrics": [
      {
        "answer_relevance": 0.8,
        "record_id": "f5a41bb9-3382-4c86-b3a8-0d8b58086bfd"
      },
      {
        "answer_relevance": 0.8,
        "record_id": "d72b655a-54c8-4d72-bf47-7d3f0c80ea6c"
      },
      {
        "answer_relevance": 0.8,
        "record_id": "572e693e-c538-4ff0-ac06-5c590dbf02aa"
      },
      {
        "answer_relevance": 1.0,
        "record_id": "81af334a-a9d5-4bb7-8927-a18063018556"
      },
      {
        "answer_relevance": 0.8,
        "record_id": "6798531e-bd16-41a0-bd07-73b04ace3eb5"
      },
      {
        "answer_relevance": 0.4,
        "record_id": "476259c3-c8ac-4eb6-b595-4f5a1ab119e5"
      }
    ]
  },
  "answer_similarity": {
    "metric_value": 0.9667,
    "mean": 0.9667,
    "min": 0.8,
    "max": 1.0,
    "total_records": 6,
    "record_level_metrics": [
      {
   

## Step 7 - Display the results <a id="results"></a>

### Get metric results for all records

In [25]:
results_df = data.copy()

for metric_name, metric_data in metrics_result.items():
    vals = {}
    if "record_level_metrics" in metric_data:
        for rm in metric_data["record_level_metrics"]:
            for m, mv in rm.items():
                if m != "record_id" and m!= "faithfulness_attributions" and m!= "context_relevances":  # Excluding columns
                    if m in vals:
                        vals[m].append(mv)
                    else:
                        vals[m] = [mv]

    if vals:
        for k, v in vals.items():
            results_df[k] = v

results_df

,context1,context2,context3,context4,question,answer,answer_relevance,answer_similarity,faithfulness,context_relevance,retrieval_precision,average_precision,reciprocal_rank,hit_rate,ndcg
0,"Last month, I announced our plan to supercharg...",We’re also ready with anti-viral treatments. I...,For that purpose we’ve mobilized American grou...,"If you travel 20 miles east of Columbus, Ohio,...",What is ARPA-H?,"\n\nARPA-H, or the Advanced Research Projects ...",0.8,1.0,1.0,1.0,0.25,1.0,1.000000,1,1.0000
1,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is the investment of Ford and GM to build...,\n\nFord is investing $11 billion to build ele...,0.8,1.0,0.8,1.0,0.25,1.0,1.000000,1,1.0000
2,My plan will cut the cost in half for most fam...,We got more than 130 countries to agree on a g...,And unlike the $2 Trillion tax cut passed in t...,We’re going after the criminals who stole bill...,What is the proposed tax rate for corporations?,\n\nThe proposed tax rate for corporations is ...,0.8,1.0,0.8,1.0,0.25,1.0,1.000000,1,1.0000
3,"If you travel 20 miles east of Columbus, Ohio,...",So let’s not wait any longer. Send it to my de...,When we use taxpayer dollars to rebuild Americ...,It is going to transform America and put us on...,What is Intel going to build?,\n\nIntel is going to build up to eight state-...,1.0,1.0,1.0,1.0,0.25,1.0,1.000000,1,1.0000
4,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",When we use taxpayer dollars to rebuild Americ...,"As Ohio Senator Sherrod Brown says, “It’s time...",How many new manufacturing jobs are created la...,"\n\n369,000 new manufacturing jobs were create...",0.8,1.0,1.0,1.0,0.25,1.0,1.000000,1,1.0000
5,So let’s not wait any longer. Send it to my de...,"If you travel 20 miles east of Columbus, Ohio,...",It is going to transform America and put us on...,Vice President Harris and I ran for office wit...,How many electric vehicle charging stations ar...,\n\nThe text does not provide specific informa...,0.4,0.8,0.4,1.0,0.25,0.3,0.333333,1,0.7077


Author: <a href="mailto:pvemulam@in.ibm.com">Pratap Kishore Varma V</a>

Copyright © 2024 IBM.